In [ ]:
let testInput = @"[1,1,3,1,1]
[1,1,5,1,1]

[[1],[2,3,4]]
[[1],4]

[9]
[[8,7,6]]

[[4,4],4,4]
[[4,4],4,4,4]

[7,7,7,7]
[7,7,7]

[]
[3]

[[[]]]
[[]]

[1,[2,[3,[4,[5,6,7]]]],8,9]
[1,[2,[3,[4,[5,6,0]]]],8,9]".Split("\n") |> Array.filter (not << String.IsNullOrEmpty)

testInput

In [ ]:
#r "nuget:FParsec"
open FParsec

In [ ]:
[<StructuredFormatDisplay("{AsString}")>]
type Packet =
  | Int of int
  | Packet of Packet list
  with
    override this.ToString() = 
      match this with 
      | Int i -> string i
      | Packet p -> sprintf "[%s]" (Seq.map (sprintf "%A") p |> String.concat ",")
    member this.AsString = this.ToString()

Formatter.SetPreferredMimeTypesFor(typeof<obj>, "text/plain")
Formatter.Register(fun (x:obj) (writer: TextWriter) -> fprintfn writer "%120A" x)

let ppacket, pPacket = createParserForwardedToRef()
do pPacket := 
    ((pint32 |>> Int) <|> ppacket) |> sepBy <| (pchar ',')
    |> between (pchar '[') (pchar ']')
    |>> Packet

let parsePacket s = match run ppacket s with Success(x,_,_) -> x | Failure(e,_,_) -> failwith "e"

let rec comparePacket x y = 
  match x,y with
  | Int i, Int j -> compare i j
  | Int _ as i, p -> comparePacket (Packet [i]) p
  | p, (Int _ as i) -> comparePacket p (Packet [i])
  | Packet [], Packet [] -> 0
  | Packet [], Packet (_ :: _) -> -1
  | Packet (_ :: _), Packet [] -> 1
  | Packet (l :: restL), Packet (r :: restR) -> 
    match comparePacket l r with 0 -> comparePacket (Packet restL) (Packet restR) | x -> x

testInput
|> Seq.map parsePacket
|> Seq.chunkBySize 2
|> Seq.map (fun [|x;y|] -> sprintf "%A %c %A" x (match comparePacket x y with 0 -> '=' | x when x < 0 -> '<' | _ -> '>') y)
|> Seq.toList

In [ ]:
let packets = 
  File.ReadAllLines "inputs/13/input.txt"
  |> Seq.except [""]
  |> Seq.map parsePacket
  |> Seq.toList

packets
|> Seq.chunkBySize 2
|> Seq.indexed
|> Seq.filter (fun (_,[|x;y|]) -> comparePacket x y <> 1)
|> Seq.sumBy (fst >> (+) 1)

In [ ]:
let dividerPackets = [parsePacket "[[2]]"; parsePacket "[[6]]"]

let allPackets = 
  packets @ dividerPackets
  |> List.sortWith comparePacket

dividerPackets
|> Seq.map (fun d -> List.findIndex ((=) d) allPackets + 1)
|> Seq.reduce (*)